In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from pyspark.sql.functions import col, signum

spark = SparkSession.builder.master("local[1]").getOrCreate()


In [10]:
#Need to carry over from day to day
#Dollar bars
df=spark.read.parquet("A_Candlestick_part.parquet")
df.show(5)


+------+------+------+------+------+------+----------+-----------------+
|Ticker|  Open|  High|   Low| Close|Volume|      Time|__index_level_0__|
+------+------+------+------+------+------+----------+-----------------+
|    AG| 10.38| 10.38| 10.38| 10.38| 140.0|1566513600|         18051935|
|  AVYA|14.715| 14.75|14.705|14.705|1553.0|1591037040|          2235656|
|   ALC|51.215|51.215| 51.18| 51.21|1656.0|1586990160|         15178518|
|   AXL| 10.02| 10.02| 10.02| 10.02| 219.0|1575679980|          1538479|
|  AQUA|  18.8| 18.81|  18.8| 18.81|1191.0|1575936780|          8339089|
+------+------+------+------+------+------+----------+-----------------+
only showing top 5 rows



In [4]:
import sys
df=df.withColumn('DolExch',col('Close')*col('Volume'))
df.select("DolExch").distinct().show(5)
df=df.withColumn("CumDolExch",f.sum("DolExch").over(Window.partitionBy('Ticker').orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show()
df.select("CumDolExch").distinct().show(5)
df=df.withColumn('DolBars', col('CumDolExch')%(1E7))#Take data pt every 10M dollars exchanged per stock
df=df.withColumn('Mark',f.when(df.DolBars<=df.DolExch,1).otherwise(0))
df.select("Mark").distinct().show()

+------------------+
|           DolExch|
+------------------+
|             2.057|
|            2.3999|
|             4.752|
|6.9998000000000005|
|             12.32|
+------------------+
only showing top 5 rows

+------+--------+-------+--------+--------+------+----------+-----------------+------------------+------------------+
|Ticker|    Open|   High|     Low|   Close|Volume|      Time|__index_level_0__|           DolExch|        CumDolExch|
+------+--------+-------+--------+--------+------+----------+-----------------+------------------+------------------+
|  ALXN|  125.02| 125.02|  125.01|  125.01| 961.0|1562627400|         14568288|         120134.61|         120134.61|
|  ALXN|  125.03| 125.08|  124.95|  125.08|3407.0|1562627460|         14568289|         426147.56|         546282.17|
|  ALXN|  125.05| 125.19|  125.05|125.1841|3887.0|1562627520|         14568290|       486590.5967|      1032872.7667|
|  ALXN|  125.15| 125.21|  125.07|  125.21|5527.0|1562627580|         14568291

In [5]:
df=df.withColumn('CumMark',f.sum("Mark").over(Window.partitionBy('Ticker').orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show(20)
df.select("CumMark").distinct().show(10)

+------+--------+-------+--------+--------+------+----------+-----------------+------------------+------------------+------------------+----+-------+
|Ticker|    Open|   High|     Low|   Close|Volume|      Time|__index_level_0__|           DolExch|        CumDolExch|           DolBars|Mark|CumMark|
+------+--------+-------+--------+--------+------+----------+-----------------+------------------+------------------+------------------+----+-------+
|  ALXN|  125.02| 125.02|  125.01|  125.01| 961.0|1562627400|         14568288|         120134.61|         120134.61|         120134.61|   1|      1|
|  ALXN|  125.03| 125.08|  124.95|  125.08|3407.0|1562627460|         14568289|         426147.56|         546282.17|         546282.17|   0|      1|
|  ALXN|  125.05| 125.19|  125.05|125.1841|3887.0|1562627520|         14568290|       486590.5967|      1032872.7667|      1032872.7667|   0|      1|
|  ALXN|  125.15| 125.21|  125.07|  125.21|5527.0|1562627580|         14568291| 692035.6699999999|  

In [6]:
df=df.withColumn("Volume", f.sum("Volume").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))

df=df.withColumn("Low",f.min("Low").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show()
print(df.head())
df=df.withColumn("Open",f.first("Open").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
print(df.head())
df=df.withColumn("Close",f.last("Close").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
df=df.filter(df.Mark==1)

#Add target vector(2 for +change, 1 for no change 0 for -change)
df=df.withColumn("next_val", f.lead(col("Close"),1).over(Window.partitionBy('Ticker').orderBy(df["Time"])))#Need to add partitionBy()
df=df.withColumn("Target", (col("next_val")-col("Close"))/col("Close"))
print(df.head())
df=df.select("Ticker","Time","Open","Low","High","Close","Volume","Target")#.withColumn("Target", 100*(col("next_val")-col("Close"))/col("Close"))
#df=df.drop("next_val").withColumn("Target",1+signum(col("Target")))
df.head()

+------+--------+--------+------+--------+--------+----------+-----------------+------------------+--------------------+------------------+----+-------+
|Ticker|    Open|    High|   Low|   Close|  Volume|      Time|__index_level_0__|           DolExch|          CumDolExch|           DolBars|Mark|CumMark|
+------+--------+--------+------+--------+--------+----------+-----------------+------------------+--------------------+------------------+----+-------+
|  AAPL|199.9402|199.9402|199.86|  199.91| 28682.0|1562628300|         25190662|        5733818.62|     2.70757898436E8| 757898.4359999895|   1|     26|
|  AAPL|  199.92|  199.95|199.86|   199.9| 61131.0|1562628360|         25190663| 6486555.100000001|     2.77244453536E8| 7244453.536000013|   0|     26|
|  AMZN| 1955.47|  1956.0|199.86|1955.555| 77913.0|1562629500|         11719451|     3.281812401E7|    6.873660628002E8|7366062.8001999855|   1|     26|
|  ACIA|    64.9| 64.9485|  64.8|  64.834|133281.0|1562694420|         23123495|  

Row(Ticker='ALXN', Time=1562627400, Open=47.61, Low=1.26, High=125.02, Close=125.01, Volume=3461073.0, Target=0.0013598912087033172)

In [7]:
#Convert Ticker value to int to include in ML algo, dictionary present (tick_dict) to convert back if needed
tickers=df.select("Ticker").distinct().rdd.flatMap(lambda x: x).collect()

tick_dict = {val : str(idx + 1) for idx, val in enumerate(tickers)} 
print(tick_dict)

from pyspark.sql.types import IntegerType
df=df.replace(to_replace=tick_dict, subset=['Ticker'])

df.printSchema()
df=df.withColumn("Ticker",col("Ticker").cast(IntegerType()))
df.printSchema()

{'ALXN': '1', 'AWAY': '2', 'AAT': '3', 'ABMD': '4', 'AEFC': '5', 'AESR': '6', 'AIV': '7', 'APM': '8', 'ARYAW': '9', 'AVY': '10', 'ARAY': '11', 'AMTX': '12', 'ARL': '13', 'AXP': '14', 'ACCO': '15', 'ARTW': '16', 'ADIL': '17', 'AVNW': '18', 'AVUV': '19', 'ALDX': '20', 'APYX': '21', 'ASH': '22', 'ARGT': '23', 'ATCX': '24', 'AHCO': '25', 'AMRHW': '26', 'AMRK': '27', 'AVEO': '28', 'AVXL': '29', 'AXU': '30', 'AEB': '31', 'ALE': '32', 'ALSN': '33', 'ASYS': '34', 'AVDL': '35', 'ACH': '36', 'AGIO': '37', 'ARA': '38', 'AQMS': '39', 'ASET': '40', 'ALG': '41', 'ASHX': '42', 'AVAL': '43', 'ALKS': '44', 'ACWX': '45', 'AEY': '46', 'AM': '47', 'AOM': '48', 'ARCT': '49', 'AYX': '50', 'AA': '51', 'ACHC': '52', 'AMRH': '53', 'ASIX': '54', 'AAPL': '55', 'AMPH': '56', 'ANDE': '57', 'AY': '58', 'ADNT': '59', 'ALL': '60', 'AOSL': '61', 'ADVM': '62', 'AMRX': '63', 'ACGLO': '64', 'ADI': '65', 'AFGC': '66', 'ALACU': '67', 'ARB': '68', 'AZZ': '69', 'AGNC': '70', 'ALTR': '71', 'AGMH': '72', 'APDN': '73', 'AEF': '

root
 |-- Ticker: string (nullable = true)
 |-- Time: long (nullable = true)
 |-- Open: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Target: double (nullable = true)

root
 |-- Ticker: integer (nullable = true)
 |-- Time: long (nullable = true)
 |-- Open: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Target: double (nullable = true)



In [8]:
from pyspark.ml.feature import VectorAssembler
#Create input features
df_cols=df.columns
df_cols=[ elem for elem in df_cols if elem not in ["Time","Target"]]
#Move features to a single vector
assembler=VectorAssembler(inputCols=df_cols,outputCol="features")
df=assembler.transform(df)
df=df.na.drop()
df=df.dropDuplicates()
print(df.count())

583540


In [9]:
df.show(10)
df=df.na.drop()
df=df.dropDuplicates()

+------+----------+---------+------+--------+--------+---------+--------------------+--------------------+
|Ticker|      Time|     Open|   Low|    High|   Close|   Volume|              Target|            features|
+------+----------+---------+------+--------+--------+---------+--------------------+--------------------+
|     1|1562693400|   199.95| 21.72|   124.5|  123.68|1327968.0|0.011238680465717871|[1.0,199.95,21.72...|
|     1|1562880300|  199.935| 21.51|  120.75|   120.7|2224125.0|-2.48550124275071...|[1.0,199.935,21.5...|
|     1|1564517880|  1989.28| 49.58|  114.81|  114.76| 831781.0|-0.00660073196235...|[1.0,1989.28,49.5...|
|     1|1564703700|  1989.35| 50.48|  114.85|  114.64| 784528.0|9.595254710397718E-4|[1.0,1989.35,50.4...|
|     1|1565391180|2013.0727|46.705|   109.2| 109.155|1044949.0|0.007558059639961549|[1.0,2013.0727,46...|
|     1|1566510540|  2021.53| 47.72|   123.4|  122.06|1063452.0|0.003604784532197...|[1.0,2021.53,47.7...|
|     1|1566841260|  2020.81|45.255| 

In [ ]:
from pyspark.ml.regression  import RandomForestRegressor
import time




train,test=df.randomSplit([0.8,0.2],seed=1)
train.show(10)
print('repartitioning')
train=train.repartition(100)
test=test.repartition(100)
print('End repartition')

start_time=time.time()

dt=RandomForestRegressor(featuresCol='features',
                         labelCol='Target',
                         maxDepth=30,
                         minInstancesPerNode=2)
print('Start training')
dtModel=dt.fit(train)
print('End training')
predictions=dtModel.transform(test)
end_time=time.time()

delta_time = end_time - start_time

# 5. print total run time 
print(f'run-time: {round(delta_time, 2)}')

predictions.printSchema()

#predictions.select('Time','Target','probability','prediction').show()

+------+----------+---------+------+--------+--------+---------+--------------------+--------------------+
|Ticker|      Time|     Open|   Low|    High|   Close|   Volume|              Target|            features|
+------+----------+---------+------+--------+--------+---------+--------------------+--------------------+
|     1|1562693400|   199.95| 21.72|   124.5|  123.68|1327968.0|0.011238680465717871|[1.0,199.95,21.72...|
|     1|1562880300|  199.935| 21.51|  120.75|   120.7|2224125.0|-2.48550124275071...|[1.0,199.935,21.5...|
|     1|1564517880|  1989.28| 49.58|  114.81|  114.76| 831781.0|-0.00660073196235...|[1.0,1989.28,49.5...|
|     1|1564703700|  1989.35| 50.48|  114.85|  114.64| 784528.0|9.595254710397718E-4|[1.0,1989.35,50.4...|
|     1|1566510540|  2021.53| 47.72|   123.4|  122.06|1063452.0|0.003604784532197...|[1.0,2021.53,47.7...|
|     1|1566841260|  2020.81|45.255|  110.81|110.7325|1024557.0|-0.00300273180863...|[1.0,2020.81,45.2...|
|     1|1570038900|2011.4897| 47.84| 

In [ ]:
#from pyspark.ml.tuning import CrossValidator


In [ ]:
max=predictions.agg({"prediction":"max"}).collect()[0]
max=max["max(prediction)"]#Max predicted increase

toDisplay=predictions.select("Time","Target","prediction")
toDisplay=toDisplay.withColumn("ToInvest",f.when(col('prediction')>0,1000.*col("prediction")/max).otherwise(0))
toDisplay=toDisplay.withColumn("Revenue", col("ToInvest")*col("Target"))
#toDisplay=toDisplay.withColumn("Target",f.round(100*col("Target"),5))
#toDisplay=toDisplay.withColumn("prediction",f.round(100*col("prediction"),5))
toDisplay.show()
print(predictions.count())

In [ ]:
profit_row=toDisplay.agg({"Revenue":"sum"}).collect()[0]

profit=profit_row["sum(Revenue)"]
print(profit)

toDisplay.filter(col("ToInvest")>0).show()

In [5]:
df=df.repartition(1000)

In [6]:
df.write.parquet("A_Candlestick")

Py4JJavaError: An error occurred while calling o43.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at org.apach

In [7]:
df.show(5)

Py4JJavaError: An error occurred while calling o42.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 26, 192.168.0.5, executor driver): java.io.FileNotFoundException: File file:/Users/grantjensen/Backtesting/A_Candlestick.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$